In [ ]:
{
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# 📊 Análise Exploratória de Dados (EDA) - Olist E-commerce\n",
                "\n",
                "**Objetivo:** Compreender a distribuição dos dados, identificar outliers, analisar correlações e extrair insights preliminares para guiar a modelagem e engenharia de dados.\n",
                "\n",
                "**Dataset:** Brazilian E-Commerce Public Dataset by Olist",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Configurações Iniciais\n",
                "import pandas as pd\n",
                "import numpy as np\n",
                "import matplotlib.pyplot as plt\n",
                "import seaborn as sns\n",
                "import os\n",
                "\n",
                "# Configuração Visual\n",
                'sns.set_theme(style="whitegrid")\n',
                "plt.rcParams['figure.figsize'] = (12, 6)\n",
                "\n",
                "# Caminho dos Dados (Relativo à pasta do notebook)\n",
                'DATA_DIR = "../../01_base_dados"',
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 1. Carregamento e Unificação dos Dados\n",
                "Vamos carregar os principais datasets para montar uma visão unificada de Pedidos + Itens + Produtos.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "try:\n",
                "    # Carregando Datasets\n",
                "    df_items = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_items_dataset.csv'))\n",
                "    df_orders = pd.read_csv(os.path.join(DATA_DIR, 'olist_orders_dataset.csv'))\n",
                "    df_products = pd.read_csv(os.path.join(DATA_DIR, 'olist_products_dataset.csv'))\n",
                "\n",
                "    # Merge (Unificação)\n",
                "    df = df_items.merge(df_orders, on='order_id', how='left')\n",
                "    df = df.merge(df_products, on='product_id', how='left')\n",
                "\n",
                "    # Conversão de Datas\n",
                "    df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])\n",
                "\n",
                '    print(f"✅ Dataset Unificado: {df.shape[0]} linhas e {df.shape[1]} colunas.")\n',
                "    display(df.head())\n",
                "except FileNotFoundError:\n",
                '    print("❌ Erro: Arquivos não encontrados. Verifique o caminho em DATA_DIR.")',
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 2. Estatísticas Descritivas\n",
                "Análise inicial de tendências centrais, dispersão e tipos de dados.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Resumo Estatístico das Variáveis Numéricas\n",
                "display(df[['price', 'freight_value', 'product_weight_g', 'product_photos_qty']].describe())",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Verificação de Nulos\n",
                "missing = df.isnull().sum()\n",
                "missing = missing[missing > 0].sort_values(ascending=False)\n",
                "if not missing.empty:\n",
                "    plt.figure(figsize=(10, 4))\n",
                '    sns.barplot(x=missing.values, y=missing.index, palette="viridis")\n',
                '    plt.title("Contagem de Valores Nulos por Coluna")\n',
                '    plt.xlabel("Qtd Nulos")\n',
                "    plt.show()\n",
                "else:\n",
                '    print("Dataset sem valores nulos!")',
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 3. Identificação de Outliers\n",
                "O preço dos produtos e o valor do frete variam drasticamente? Vamos usar Boxplots para visualizar.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "fig, ax = plt.subplots(1, 2, figsize=(16, 6))\n",
                "\n",
                "# Boxplot Preço\n",
                "sns.boxplot(x=df['price'], ax=ax[0], color='skyblue')\n",
                "ax[0].set_title('Distribuição de Preços (Com Outliers)')\n",
                "ax[0].set_xlabel('Preço (R$)')\n",
                "\n",
                "# Boxplot Frete\n",
                "sns.boxplot(x=df['freight_value'], ax=ax[1], color='salmon')\n",
                "ax[1].set_title('Distribuição de Frete')\n",
                "ax[1].set_xlabel('Frete (R$)')\n",
                "\n",
                "plt.show()",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Zoom na distribuição (Removendo outliers visuais extremos para análise)\n",
                "limite_preco = df['price'].quantile(0.95)\n",
                "plt.figure(figsize=(10, 5))\n",
                "sns.histplot(df[df['price'] < limite_preco]['price'], bins=50, kde=True)\n",
                "plt.title(f'Histograma de Preços (95% dos dados < R$ {limite_preco:.2f})')\n",
                "plt.xlabel('Preço')\n",
                "plt.show()",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 4. Análise de Correlações\n",
                "Existe relação entre o peso do produto e o valor do frete? E entre o preço e o frete?",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "cols_corr = ['price', 'freight_value', 'product_weight_g', 'product_volume_cm3', 'product_photos_qty']\n",
                "\n",
                "# Calculando Volume se não existir (aproximado)\n",
                "if 'product_length_cm' in df.columns:\n",
                "    df['product_volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']\n",
                "\n",
                "# Matriz de Correlação\n",
                "corr_matrix = df[cols_corr].corr()\n",
                "\n",
                "plt.figure(figsize=(8, 6))\n",
                "sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=\".2f\", linewidths=0.5)\n",
                "plt.title('Matriz de Correlação (Heatmap)')\n",
                "plt.show()",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "**Insight:** Normalmente espera-se uma correlação positiva forte entre `product_weight_g` (peso) e `freight_value` (frete)."
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 5. Análise de Categorias (Pareto)\n",
                "Quais categorias representam a maior parte da receita?",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "receita_categoria = df.groupby('product_category_name')['price'].sum().sort_values(ascending=False).head(10)\n",
                "\n",
                "plt.figure(figsize=(12, 6))\n",
                "sns.barplot(x=receita_categoria.values, y=receita_categoria.index, palette='magma')\n",
                "plt.title('Top 10 Categorias por Receita')\n",
                "plt.xlabel('Receita Total (R$)')\n",
                "plt.ylabel('Categoria')\n",
                "plt.show()",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 6. Conclusões Preliminares\n",
                "\n",
                "1. **Outliers:** Identificamos produtos de alto valor que distorcem a média. A mediana é uma métrica mais segura para 'preço típico'.\n",
                "2. **Data Quality:** Existem valores nulos em categorias e pesos que precisam ser tratados na camada Silver.\n",
                "3. **Correlação:** O frete tem correlação com o peso, validando a lógica logística.\n",
                "4. **Sazonalidade:** (Pode ser explorada na análise temporal detalhada no Dashboard).",
            ],
        },
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3",
        },
        "language_info": {
            "codemirror_mode": {"name": "ipython", "version": 3},
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.8.5",
        },
    },
    "nbformat": 4,
    "nbformat_minor": 4,
}